In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('../data/raw/combined_data.csv')

In [9]:
df.head(10)

,0,1,2,3
0,05-20-1991,08:00,58,101
1,05-20-1991,08:00,33,5
2,05-20-1991,08:00,34,27
3,05-20-1991,12:00,60,89
4,05-20-1991,12:00,33,3
5,05-20-1991,18:00,62,94
6,05-20-1991,18:00,33,6
7,05-20-1991,22:00,48,109
8,05-21-1991,08:00,58,174
9,05-21-1991,08:00,33,5
